In [3]:
import pandas as pd
import numpy as np

df_pred = pd.read_csv('./pred.csv')
df_test = pd.read_csv('./test.csv')

df_pred_new = pd.merge(
    df_pred, 
    df_test.loc[:, ['user_id', 'item_id', 'relevance']], on=['user_id', 'item_id'], how='left'
)

df_pred_new = df_pred_new[df_pred_new.user_id.isin(df_test.user_id)]

df_pred_new.fillna(0, inplace=True)

df_pred_new.loc[df_pred_new.relevance != 0, 'relevance'] = 1

df_pred_new.relevance.value_counts()

0.0    51516
1.0     3144
Name: relevance, dtype: int64

In [4]:
df_pred_new

,user_id,item_id,score,relevance
40,3,590,0.280305,0.0
41,3,1136,0.263707,0.0
42,3,920,0.231234,0.0
43,3,1246,0.188023,0.0
44,3,318,0.187111,0.0
...,...,...,...,...
1179555,71534,1304,0.088851,0.0
1179556,71534,778,0.085875,0.0
1179557,71534,1307,0.082908,0.0
1179558,71534,1258,0.082717,0.0


In [42]:
df_pred_new.relevance.value_counts()

0.0    51516
1.0     3144
Name: relevance, dtype: int64

In [7]:
df_pred_new['ones'] = 1

In [57]:
df_pred

,user_id,item_id,score
0,1,500,0.519073
1,1,595,0.514183
2,1,344,0.508895
3,1,457,0.435809
4,1,596,0.409035
...,...,...,...
1180115,71567,1580,0.109929
1180116,71567,1197,0.109768
1180117,71567,356,0.109597
1180118,71567,1387,0.105217


In [55]:
df_test

,user_id,item_id,rating,timestamp,relevance
0,1256,5378,5.0,2007-10-31 12:18:24,5.0
1,1256,778,4.5,2007-10-31 12:19:51,4.5
2,1256,8376,5.0,2007-10-31 12:30:59,5.0
3,1256,2594,5.0,2007-10-31 12:20:49,5.0
4,1256,30810,4.5,2007-10-31 12:32:24,4.5
...,...,...,...,...,...
38806,68571,112,5.0,2007-01-28 01:37:59,5.0
38807,68571,7256,5.0,2007-01-28 01:36:16,5.0
38808,68571,34405,4.5,2007-01-28 01:13:53,4.5
38809,68571,2618,5.0,2007-01-28 01:43:19,5.0


In [128]:
df = pd.merge(df_test, df_pred, how='outer', on=['user_id', 'item_id'])

In [123]:
df

,user_id,item_id,rating,timestamp,relevance,score
0,1256,5378,5.0,2007-10-31 12:18:24,5.0,NaN
1,1256,778,4.5,2007-10-31 12:19:51,4.5,NaN
2,1256,8376,5.0,2007-10-31 12:30:59,5.0,NaN
3,1256,2594,5.0,2007-10-31 12:20:49,5.0,NaN
4,1256,30810,4.5,2007-10-31 12:32:24,4.5,NaN
...,...,...,...,...,...,...
1215782,71567,1580,NaN,NaN,NaN,0.109929
1215783,71567,1197,NaN,NaN,NaN,0.109768
1215784,71567,356,NaN,NaN,NaN,0.109597
1215785,71567,1387,NaN,NaN,NaN,0.105217


In [124]:
df.score.isna() & df.user_id.isin(df_pred.user_id)

0           True
1           True
2           True
3           True
4           True
           ...  
1215782    False
1215783    False
1215784    False
1215785    False
1215786    False
Length: 1215787, dtype: bool

In [129]:
df = df[df.score.isna() & df.user_id.isin(df_pred.user_id)]

In [130]:
df.score = 0

In [91]:
df

,user_id,item_id,rating,timestamp,relevance,score
0,1256,5378,5.0,2007-10-31 12:18:24,5.0,0
1,1256,778,4.5,2007-10-31 12:19:51,4.5,0
2,1256,8376,5.0,2007-10-31 12:30:59,5.0,0
3,1256,2594,5.0,2007-10-31 12:20:49,5.0,0
4,1256,30810,4.5,2007-10-31 12:32:24,4.5,0
...,...,...,...,...,...,...
38806,68571,112,5.0,2007-01-28 01:37:59,5.0,0
38807,68571,7256,5.0,2007-01-28 01:36:16,5.0,0
38808,68571,34405,4.5,2007-01-28 01:13:53,4.5,0
38809,68571,2618,5.0,2007-01-28 01:43:19,5.0,0


In [47]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
from collections import defaultdict

# from surprise import Dataset
# from surprise import SVD
# from surprise.model_selection import KFold


def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

In [34]:
next(df_pred_new.loc[:, ['user_id', 'item_id', 'relevance', 'score', 'ones']].iterrows())[1]

user_id        3.000000
item_id      590.000000
relevance      0.000000
score          0.280305
ones           1.000000
Name: 40, dtype: float64

In [92]:
preds = [x[1] for x in 
         pd.concat([df_pred_new, df]).loc[:, ['user_id', 'item_id', 'relevance', 'score', 'ones']].iterrows()]


In [107]:
res = precision_recall_at_k(
    preds,
    k=20,
    threshold=0.00000000000000000000000000000001
)

In [108]:
np.mean(list(res[0].values()))

0.057519209659714604

In [109]:
np.mean(list(res[1].values()))

0.09632131280310642